<a href="https://colab.research.google.com/github/DionKimmm/2019SummerML/blob/master/Day18_mnist_using_tf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# 그래프가 왜 중요한가
# 왜 GPU를 쓰는가
# 일을 쪼개면 성능 좋은 코어 보다는 많은 코어가 좋다
# CPU는 8개의 코어, GPU는 몇천개의 코어
# 미분으로 학습을 시키니까 오래걸린다
# 다른 방법으로 보완 -> back propagation

## Tensorflow를 이용한 MNIST Classification

In [0]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)
import tensorflow as tf

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [0]:
import warnings
warnings.filterwarnings('ignore')

In [0]:
#Settings
learning_rate = 0.001
training_epochs = 10
batch_size = 100
display_step = 1

In [0]:
#NN Parameters
n_hidden_1 = 256 # Layer 1 has 256 neurons
n_hidden_2 = 256 # Layer 2 has 256 neurons
n_hidden_3 = 256 # Layer 3 has 256 neurons
n_input = 784 # Input is 28*28 = 784
n_classes = 10 # 10 possible outputs (0-9 digits)

In [0]:
x = tf.placeholder("float", [None, n_input])
y = tf.placeholder("float", [None, n_classes])

In [0]:
# Function to create a 3 layer perceptron using relu activation
def generate_three_layer_perceptron(x, weights, biases):
	layer_1 = tf.add(tf.matmul(x, weights['h1']), biases['b1'])
	layer_1 = tf.nn.relu(layer_1)

	layer_2 = tf.add(tf.matmul(layer_1, weights['h2']), biases['b2'])
	layer_2 = tf.nn.relu(layer_2)

	layer_3 = tf.add(tf.matmul(layer_2, weights['h3']), biases['b3'])
	layer_3 = tf.nn.relu(layer_3)

	out_layer = tf.matmul(layer_3, weights['out']) + biases['out']
	return out_layer

In [0]:
weights = {
	'h1': tf.Variable(tf.random_normal([n_input, n_hidden_1])),
	'h2': tf.Variable(tf.random_normal([n_hidden_1, n_hidden_2])),
	'h3': tf.Variable(tf.random_normal([n_hidden_2, n_hidden_3])),
	'out': tf.Variable(tf.random_normal([n_hidden_3, n_classes]))
}
biases = {
	'b1': tf.Variable(tf.random_normal([n_hidden_1])),
	'b2': tf.Variable(tf.random_normal([n_hidden_2])),
	'b3': tf.Variable(tf.random_normal([n_hidden_3])),
	'out': tf.Variable(tf.random_normal([n_classes]))
}

In [0]:
# Create the perceptron
pred = generate_three_layer_perceptron(x, weights, biases)


In [0]:
# Cost function is avg softmax
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)


In [0]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

In [0]:
init = tf.global_variables_initializer()
saver = tf.train.Saver()

with tf.Session() as sess:
	sess.run(init)

	for epoch in range(training_epochs):
		avg_cost = 0.
		total_batch = int(mnist.train.num_examples/batch_size)
		
		for i in range(total_batch):
			batch_x, batch_y = mnist.train.next_batch(batch_size)
			_, c = sess.run([optimizer, cost], feed_dict={x: batch_x, y: batch_y})
			avg_cost += c/total_batch

		if epoch % display_step == 0:
			print("Epoch:", '%04d' % (epoch+1), "cost=", "{:.9f}".format(avg_cost))

	print("Optimization Finished!")
	

	print("Beginning evaluation of model...")

	correct_prediction = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
	accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
	print("Accuracy:", accuracy.eval({x: mnist.test.images, y: mnist.test.labels}))



Epoch: 0001 cost= 1304.081013544
Epoch: 0002 cost= 339.329752072
Epoch: 0003 cost= 208.180766054
Epoch: 0004 cost= 138.337498103
Epoch: 0005 cost= 96.135323123
Epoch: 0006 cost= 68.163112141
Epoch: 0007 cost= 49.396004987
Epoch: 0008 cost= 34.806565150
Epoch: 0009 cost= 26.791991973
Epoch: 0010 cost= 19.342596144
Optimization Finished!
Beginning evaluation of model...
Accuracy: 0.9361
